**Introduction**

Multi-source domain adaptation (MDA) is a fascinating area in machine learning that aims to transfer knowledge from multiple source domains to an unlabeled target domain.

In MDA, the goal is to leverage the information from all available source domains to improve the performance of the model on the target domain.

Ensemble

Ensemble methods work because they combine the strengths of multiple models to produce a more robust and accurate prediction or decision than any single model could achieve on its own

Source-Specific Models

Source-Specific Models: Train individual models for each source domain and combine their predictions for the target domain using ensemble techniques like weighted averaging or majority voting

Imports

In [1]:
# !pip install torchensemble

In [2]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder

from torchensemble import VotingClassifier
from torchensemble.utils.logging import set_logger

Data Processing

In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("mei1963/domainnet")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/mei1963/domainnet/versions/1


In [4]:
import os
def walk_through_dir(dir_path):
  """
  Walks through dir_path returning its contents.
  Args:
    dir_path (str or pathlib.Path): target directory

  Returns:
    A print out of:
      number of subdiretories in dir_path
      number of images (files) in each subdirectory
      name of each subdirectory
  """
  for dirpath, dirnames, filenames in os.walk(dir_path):
    print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

# walk_through_dir(path)

In [5]:
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [6]:
simple_transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),

    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [7]:
real = os.path.join(path, "DomainNet/real")
sketch = os.path.join(path, "DomainNet/sketch")
clip_art = os.path.join(path, "DomainNet/clipart")


In [8]:
real_dataset = ImageFolder(real, transform=transform)
sketch_dataset = ImageFolder(sketch, transform=transform)
clipart_dataset = ImageFolder(clip_art, transform=transform)


In [9]:
real_dataloader = DataLoader(real_dataset, batch_size=32, shuffle=True)
sketch_dataloader = DataLoader(sketch_dataset, batch_size=32, shuffle=True)
clipart_dataloader = DataLoader(clipart_dataset, batch_size=32, shuffle=True)

In [10]:
len(real_dataset), len(sketch_dataset), len(clipart_dataset)

(175327, 70386, 48833)

Models

In [38]:
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=345):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 64, 3, padding=1)
        self.conv2 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv3 = nn.Conv2d(128, 256, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(256 * 4 * 4, 512)
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

Initialize the Ensemble

In [39]:
ensemble = VotingClassifier(
    estimator = SimpleCNN,
    n_estimators = 3,
    cuda = torch.cuda.is_available(),
    voting_strategy= "hard",
    estimator_args={"num_classes": 345}
)

In [40]:
# Set the loss and optimizer
criterion = nn.CrossEntropyLoss()
ensemble.set_criterion(criterion)
ensemble.set_optimizer("Adam", lr=0.0001)

In [41]:
# List of DataLoaders for different domains
dataloaders = [real_dataloader, sketch_dataloader, clipart_dataloader]

In [42]:
img, label = next(iter(clipart_dataloader))

# Batch size will now be 1, try changing the batch_size parameter above and see what happens
print(f"Image shape: {img.shape} -> [batch_size, color_channels, height, width]")
print(f"Label shape: {label.shape}")

Image shape: torch.Size([32, 3, 32, 32]) -> [batch_size, color_channels, height, width]
Label shape: torch.Size([32])


In [43]:
# Set the logger to print out training details
set_logger()

<RootLogger root (DEBUG)>

In [44]:
ensemble.fit

<bound method VotingClassifier.fit of VotingClassifier(
  (estimators_): ModuleList()
  (_criterion): CrossEntropyLoss()
)>

Train on Multiple Source Domains Sequentially

In [ ]:
def train_on_multiple_domains(ensemble, dataloaders, epochs=5):
    for domain_idx, dataloader in enumerate(dataloaders):
        print(f"Training on Domain {domain_idx}...")
        ensemble.fit(dataloader, epochs=epochs, log_interval=1000)
        print(f"Finished training on Domain {domain_idx}.")



# Train on multiple source domains
train_on_multiple_domains(ensemble, dataloaders, epochs=3)



Training on Domain 0...
Estimator: 000 | Epoch: 000 | Batch: 000 | Loss: 5.84377 | Correct: 0/32
Estimator: 000 | Epoch: 000 | Batch: 1000 | Loss: 4.93720 | Correct: 1/32
Estimator: 000 | Epoch: 000 | Batch: 2000 | Loss: 4.41009 | Correct: 4/32
Estimator: 000 | Epoch: 000 | Batch: 3000 | Loss: 4.16343 | Correct: 4/32
Estimator: 000 | Epoch: 000 | Batch: 4000 | Loss: 5.06649 | Correct: 1/32
Estimator: 000 | Epoch: 000 | Batch: 5000 | Loss: 4.40790 | Correct: 3/32
Estimator: 001 | Epoch: 000 | Batch: 000 | Loss: 5.85139 | Correct: 0/32
Estimator: 001 | Epoch: 000 | Batch: 1000 | Loss: 4.52578 | Correct: 4/32
Estimator: 001 | Epoch: 000 | Batch: 2000 | Loss: 4.59016 | Correct: 4/32
Estimator: 001 | Epoch: 000 | Batch: 3000 | Loss: 4.16539 | Correct: 6/32
Estimator: 001 | Epoch: 000 | Batch: 4000 | Loss: 4.06004 | Correct: 5/32
Estimator: 001 | Epoch: 000 | Batch: 5000 | Loss: 3.78441 | Correct: 4/32
Estimator: 002 | Epoch: 000 | Batch: 000 | Loss: 5.84166 | Correct: 0/32
Estimator: 002 | 

2025-03-08 05:10:40,744 - INFO: Saving the model to `./VotingClassifier_SimpleCNN_3_ckpt.pth`


Estimator: 000 | Epoch: 001 | Batch: 000 | Loss: 3.66082 | Correct: 7/32
Estimator: 000 | Epoch: 001 | Batch: 1000 | Loss: 4.11870 | Correct: 4/32
Estimator: 000 | Epoch: 001 | Batch: 2000 | Loss: 3.63262 | Correct: 7/32
Estimator: 000 | Epoch: 001 | Batch: 3000 | Loss: 3.54670 | Correct: 5/32
Estimator: 000 | Epoch: 001 | Batch: 4000 | Loss: 3.89510 | Correct: 4/32
Estimator: 000 | Epoch: 001 | Batch: 5000 | Loss: 3.12076 | Correct: 12/32
Estimator: 001 | Epoch: 001 | Batch: 000 | Loss: 3.51839 | Correct: 5/32
Estimator: 001 | Epoch: 001 | Batch: 1000 | Loss: 3.88210 | Correct: 4/32
Estimator: 001 | Epoch: 001 | Batch: 2000 | Loss: 3.72030 | Correct: 8/32
Estimator: 001 | Epoch: 001 | Batch: 3000 | Loss: 3.39476 | Correct: 6/32
Estimator: 001 | Epoch: 001 | Batch: 4000 | Loss: 4.10638 | Correct: 2/32
Estimator: 001 | Epoch: 001 | Batch: 5000 | Loss: 3.81811 | Correct: 6/32
Estimator: 002 | Epoch: 001 | Batch: 000 | Loss: 3.83399 | Correct: 7/32
Estimator: 002 | Epoch: 001 | Batch: 100

2025-03-08 05:39:59,768 - INFO: Saving the model to `./VotingClassifier_SimpleCNN_3_ckpt.pth`


Estimator: 000 | Epoch: 002 | Batch: 000 | Loss: 3.40195 | Correct: 12/32
Estimator: 000 | Epoch: 002 | Batch: 1000 | Loss: 3.07100 | Correct: 8/32
Estimator: 000 | Epoch: 002 | Batch: 2000 | Loss: 3.83516 | Correct: 2/32
Estimator: 000 | Epoch: 002 | Batch: 3000 | Loss: 3.74249 | Correct: 5/32
Estimator: 000 | Epoch: 002 | Batch: 4000 | Loss: 2.41340 | Correct: 12/32
Estimator: 000 | Epoch: 002 | Batch: 5000 | Loss: 3.36919 | Correct: 7/32
Estimator: 001 | Epoch: 002 | Batch: 000 | Loss: 3.41971 | Correct: 6/32
Estimator: 001 | Epoch: 002 | Batch: 1000 | Loss: 3.15194 | Correct: 5/32


In [ ]:
# print(f"Number of estimators: {len(ensemble.estimators_)}")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Define the path where you want to save the model
save_path = "/content/drive/MyDrive/ensemble/model.pth"  # You can change this to your desired path



# Ensure the directory exists
os.makedirs(os.path.dirname(save_path), exist_ok=True)

# Save the entire ensemble model
torch.save(ensemble, save_path)


Test Loop

In [ ]:
# Test path
quickdraw = os.path.join(path, "DomainNet/quickdraw")

# Test dataset
quickdraw_dataset = ImageFolder(quickdraw, transform=transform)

# Test dataloader
quickdraw_dataloader = DataLoader(quickdraw_dataset, batch_size=32, shuffle=True)

In [ ]:
# Evaluate the model on the 4th dataset
def evaluate_on_new_data(ensemble, dataloader):
    ensemble.eval()  # Set the model to evaluation mode

    correct = 0
    total = 0
    running_loss = 0.0

    with torch.no_grad():  # Disable gradient computation for evaluation
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(ensemble.device), labels.to(ensemble.device)

            outputs = ensemble(inputs)
            loss = ensemble._criterion(outputs, labels)
            running_loss += loss.item()

            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    accuracy = 100 * correct / total
    avg_loss = running_loss / len(dataloader)

    print(f"Loss on new dataset: {avg_loss:.3f}")
    print(f"Accuracy on new dataset: {accuracy:.2f}%")

# Evaluate on the 4th dataset
evaluate_on_new_data(ensemble, quickdraw_dataloader)

**Conclusion**

It looks like we're getting low accuracy when testing our 4th dataset (QuickDraw), and underfitting is occurring during the training loop.Here are a few options we can implement next to increase our results.

1. Increase Model Complexity
Enhance the capability of your model to capture complex patterns by increasing its depth and the number of filters.

2. Data Augmentation
Use more diverse data augmentation techniques to enrich the training data.

3. Fine-Tune on Specific Data
Fine-tune your ensemble on a portion of the QuickDraw dataset to help it adapt better.

4. Hyperparameter Tuning
Experiment with different learning rates, batch sizes, and optimizers to find the optimal configuration.

5. Regularization Techniques
Add dropout or L2 regularization to prevent overfitting and improve model robustness.